In [1]:
import tensorflow as tf
import os
import time
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 测试GPU
with tf.device('/device:GPU:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
mnist = input_data.read_data_sets("MINIST_data",one_hot=True)

batch_size = 100

n_batch = mnist.train.num_examples // batch_size

def variable_summary(var):
    """统计参数"""
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean',mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev',stddev)
    tf.summary.scalar('max',tf.reduce_max(var))
    tf.summary.scalar('min',tf.reduce_min(var))
    tf.summary.histogram('histogram',var)

#初始化权值
def weight_variable(shape, name=None):
    initial = tf.truncated_normal(shape,stddev = 0.1)
    return tf.Variable(initial, name=name)

#初始化偏置
def bias_variable(shape, name=None):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial, name=name)

#卷积层
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1], padding='SAME')

#池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1], strides = [1,2,2,1], padding='SAME')
with tf.device('/device:GPU:0'):
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    with tf.name_scope('input'):
        #placeholder
        x = tf.placeholder(tf.float32,[None,784], name='x_input')
        y = tf.placeholder(tf.float32,[None,10], name ='y_input')
        
        with tf.name_scope('x_image'):
            #reshape for image
            x_image = tf.reshape(x,[-1,28,28,1], name='x_image')

    with tf.name_scope('Conv1'):
        #初始化第一个卷积层
        # 卷积大小（5x5）通道（1）输出层
        with tf.name_scope('w_conv1'):
            w_conv1 = weight_variable([5,5,1,8], name='w_conv1')
        with tf.name_scope('b_conv1'):
            b_conv1 = bias_variable([8], name='b_conv1')

        #卷积及池化操作
        with tf.name_scope('conv2d_1'):
            conv2d_1 = conv2d(x_image,w_conv1) + b_conv1
            
        with tf.name_scope('relu'):
            h_conv1 = tf.nn.relu(conv2d_1)
        with tf.name_scope('h_pool1'):
            h_pool1 = max_pool_2x2(h_conv1) # 14x14x32

    #第二个卷积层
    with tf.name_scope('Conv2'):
        with tf.name_scope('w_conv2'):
            w_conv2 = weight_variable([5,5,8,20], name='w_conv2')
        with tf.name_scope('b_conv2'):
            b_conv2 = bias_variable([20], name='b_conv2')

        #卷积及池化操作
        with tf.name_scope('conv2d_2'):
            conv2d_2 = conv2d(h_pool1,w_conv2) + b_conv2
        with tf.name_scope('relu'):
            h_conv2 = tf.nn.relu(conv2d_2)
        with tf.name_scope('h_pool2'):
            h_pool2 = max_pool_2x2(h_conv2) # 7x7*64

    #两层处理后 结果为 7x7大小

    #初始化第一个全连接层的权值
    with tf.name_scope('fc1'):
        with tf.name_scope('w_fc1'):
            w_fc1 = weight_variable([7*7*20,500], name='w_fc1')
        with tf.name_scope('b_fc1'):
            b_fc1 = bias_variable([500], name='b_fc1')

    with tf.name_scope('h_pool2_flat'):
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*20], name='h_pool2_flat')
    with tf.name_scope('wx_plub_b1'):
        wx_plub_b1 = tf.matmul(h_pool2_flat,w_fc1) + b_fc1
    with tf.name_scope('relu'):
        h_fc1 = tf.nn.relu(wx_plub_b1)
    
    # 神经元的输出概率
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    with tf.name_scope('h_fc1_drop'):
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob, name='h_fc1_drop')

    #初始化第二个全连接层
    with tf.name_scope('fc2'):
        with tf.name_scope('w_fc2'):
            w_fc2 = weight_variable([500,10], name='w_fc2')
        with tf.name_scope('b_fc2'):
            b_fc2 = bias_variable([10], name='b_fc2')
        with tf.name_scope('wx_plus_b2'):
            wx_plus_b2 = tf.matmul(h_fc1_drop,w_fc2) + b_fc2
        with tf.name_scope('softmax'):
            prediction = tf.nn.softmax(wx_plus_b2)

    #交叉熵代价函数
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y,logits = prediction), name='cross_entropy')
        tf.summary.scalar('cross_entropy', cross_entropy)

    #优化器
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
    # 准确率
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32), name='accuracy')
            tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()

start = time.time()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
config.allow_soft_placement = True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('logs/train', sess.graph)
    test_writer = tf.summary.FileWriter('logs/test', sess.graph)
    for epoch in range(10):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict = {x:batch_xs,y:batch_ys,keep_prob:0.7})
           
            # summary = sess.run(merged, feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
            # train_writer.add_summary(summary, epoch)
            
            # 测试集
            # batch_xs,batch_ys = mnist.test.next_batch(batch_size)
            # summary = sess.run(merged, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.0})
            # test_writer.add_summary(summary, epoch)
            
        acc = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels,keep_prob:0.7})
        print("Iter " + str(epoch) + ",Testing accuracy " + str(acc))
        print(time.time() - start)

Extracting MINIST_data\train-images-idx3-ubyte.gz
Extracting MINIST_data\train-labels-idx1-ubyte.gz
Extracting MINIST_data\t10k-images-idx3-ubyte.gz
Extracting MINIST_data\t10k-labels-idx1-ubyte.gz
Iter 0,Testing accuracy 0.8931
10.909832239151001
Iter 1,Testing accuracy 0.9288
21.274123907089233
Iter 2,Testing accuracy 0.9482
42.94618463516235
Iter 3,Testing accuracy 0.9582
51.13030457496643
Iter 4,Testing accuracy 0.9649
59.58170986175537
Iter 5,Testing accuracy 0.9692
69.34062027931213
Iter 6,Testing accuracy 0.9723
79.48051142692566
Iter 7,Testing accuracy 0.9745
91.17025876045227
Iter 8,Testing accuracy 0.9762
101.59339356422424
Iter 9,Testing accuracy 0.9777
111.7751727104187


In [15]:
config = tf.ConfigProto()

In [ ]:
config.gpu_options.